<a href="https://colab.research.google.com/github/MartaGacek1/MonteCarloProject2/blob/main/MonteCarloProjekt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projekt II - Wprowadzenie do symulacji i metod Monte Carlo

## Marta Gacek

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
import pandas as pd

np.random.seed(123)

S_0 = 100
K = 100
r = 0.05
sigma = 0.25
R = 100000

## 1. Prawdziwa wartość opcji

Ta wartość zostanie wykorzystana do porównywania wyników w przypadkach gdy $n=1$ dla różnych estymatorów.

In [58]:
d1 = (np.log(S_0/K) + r + sigma**2/2)/sigma
d2 = d1 - sigma
real_exp = S_0 * sc.stats.norm.cdf(d1) - K * np.exp(-r) * sc.stats.norm.cdf(d2)
print(real_exp)

12.335998930368717


## 2. Crude Monte Carlo Estimator

### Przypadek $n=1$

In [56]:
import numpy as np

z = np.random.normal(0, 1, R)  # zmienne z rozkładu standardowego normalnego
S_T = S_0 * np.exp(r - (sigma**2)/2 + sigma * z)  # geometryczny ruch Browna
y = np.maximum(S_T - K, 0)
est_I = np.exp(-r) * np.mean(y)

print(f"Estymowana wartość opcji dla n=1 to: {est_I:.4f}")

Estymowana wartość opcji dla n=1 to: 12.2645


In [57]:
# Przedział ufności

alpha = 0.05
sig = np.var(y)
left = est_I - (sc.stats.norm.ppf(1-alpha/2) * np.sqrt(sig)/np.sqrt(R))
right = est_I + (sc.stats.norm.ppf(1-alpha/2) * np.sqrt(sig)/np.sqrt(R))
conf_interval = [left, right]
conf_interval

[12.144814230137163, 12.384230327207257]

### Przypadek $n > 1$

In [59]:
n = 50  # zrobić dla kilku różnych n
step = 1/n

z = np.random.normal(0, 1, (R, n))
prices = np.zeros((R, n))
prices[:, 0] = S_0

for t in range(1, n):
    prices[:, t] = prices[:, t-1] * np.exp((r - sigma**2/2) * step + sigma * np.sqrt(step) * z[:, t-1])  # geometryczny ruch Browna

A_n = np.mean(prices, axis=1)
y = np.maximum(A_n - K, 0)
est_I = np.exp(-r) * np.mean(y)

print(f"Estymowana wartość opcji dla n={n} wynosi: {est_I:.4f}")
#print(z)

Estymowana wartość opcji dla n=50 wynosi: 6.8087


## 3.Stratified Estimator

## 4. Antithetic Estimator

### Przypadek $n=1$

In [60]:
z = np.random.normal(0, 1, int(R))
z_anty = -z  # zmienne antytetyczne
S_T = S_0 * np.exp(r - sigma**2/2 + sigma * z)  # geometryczny ruch Browna
S_T_anty = S_0 * np.exp(r - sigma**2/2 + sigma * z_anty)
y = np.maximum(S_T - K, 0)
y_anty = np.maximum(S_T - K, 0)
est_I = np.exp(-r) * (np.mean(y) + np.mean(y_anty)) / 2  # średnia

print(f"Estymowana wartość opcji to: {est_I:.4f}")

Estymowana wartość opcji to: 12.3076


### 5. Control Variate Estimator

### Przypadek $n=1$

In [61]:
z = np.random.normal(0, 1, int(R))
S_T = S_0 * np.exp(r - sigma**2/2 + sigma * z)  # geometryczny ruch Browna
y = np.maximum(S_T - K, 0)
X = z  # zmienna kontrolna B(1), bo z ~ N(0, 1)
exp_X = 0  # znana wartość oczekiwana zmiennej B(1)
est_I_cmc = np.exp(-r) * np.mean(y)  # Y_R^{CMC} "z daszkiem", wykorzystywany we wzorze
X_R = np.mean(X)  # średnia, czyli X_R "z daszkiem"

c1 = np.cov(y, X)[0, 1]
c2 = np.var(X)
c = c1 / c2  # współczynnik c

est_I = est_I_cmc + c * (X_R - exp_X)

print(f"Estymowana wartość opcji: {est_I:.4f}")

Estymowana wartość opcji: 12.2695
